<a href="https://colab.research.google.com/github/Asritha-S/Emotion-dectection-from-text/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 EMOTION DETECTION FROM TEXT

In [ ]:
!pip3 install ktrain -q

     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 981 kB 62.2 MB/s 
     |████████████████████████████████| 263 kB 64.9 MB/s 
     |████████████████████████████████| 3.8 MB 53.5 MB/s 
     |████████████████████████████████| 1.3 MB 60.3 MB/s 
     |████████████████████████████████| 468 kB 65.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.7 MB/s 
     |████████████████████████████████| 120 kB 66.5 MB/s 
     |████████████████████████████████| 880 kB 73.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

Importing data

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_train.csv'
res = requests.get(url, allow_redirects=True)
with open('data_train.csv','wb') as file:
    file.write(res.content)
data_train = pd.read_csv('data_train.csv')

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_test.csv'
res = requests.get(url, allow_redirects=True)
with open('data_test.csv','wb') as file:
    file.write(res.content)
data_test = pd.read_csv('data_test.csv')

In [ ]:
X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...
5,sadness,When my family heard that my Mother's cousin w...
6,joy,Finding out I am chosen to collect norms for C...
7,anger,A spokesperson said : ` Glen is furious that t...
8,neutral,Yes .
9,sadness,"When I see people with burns I feel sad, actua..."


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

Data preprocessing

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


Training and validation

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
1323/1323 [==============================] - 842s 620ms/step - loss: 0.9321 - accuracy: 0.6512 - val_loss: 0.5920 - val_accuracy: 0.7937
Epoch 2/3
1323/1323 [==============================] - 818s 619ms/step - loss: 0.4714 - accuracy: 0.8361 - val_loss: 0.5344 - val_accuracy: 0.8155
Epoch 3/3
1323/1323 [==============================] - 818s 619ms/step - loss: 0.2165 - accuracy: 0.9335 - val_loss: 0.5576 - val_accuracy: 0.8243


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.87      0.84      0.85       707
     sadness       0.79      0.81      0.80       676
        fear       0.86      0.84      0.85       679
       anger       0.81      0.78      0.80       693
     neutral       0.79      0.85      0.82       638

    accuracy                           0.82      3393
   macro avg       0.82      0.82      0.82      3393
weighted avg       0.83      0.82      0.82      3393



array([[592,  20,  16,  12,  67],
       [ 14, 549,  34,  53,  26],
       [ 21,  36, 571,  38,  13],
       [ 17,  64,  33, 543,  36],
       [ 36,  22,  12,  26, 542]])

Obtained accuracy:0.82

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [ ]:
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (0.10)


In [ ]:
 predictor.save("/content/drive/MyDrive/bert_model")